# Stash codes

In [76]:
# Remember: use ceh conda environment

%load_ext autoreload
%autoreload

import iris
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set variables

For now, I'm removing the variables which I can't find in either apt4 or apt5

In [107]:
dir = '../data/'
file = 'bc179a.p41930oct.pp'
file2 = 'bc179a.p51929oct.pp'
file3 = 'bc179a.p41929oct.pp'

stash_conFIRE = {'vegcover'           : 'm01s03i317',
                 'alpha'              : 'm01s08i223',
#                'emc'                : 'm01s03i245',
#                'treeCover'          : 'm01s03i317', # same as vegcover
                 'lightning'          : 'm01s50i082',
#                'pasture'            : 'm01s00i458',
#                'population_density' : 'population_density2000-2014.nc',
                 'relative_humidity'  : 'm01s03i245'}
#                'cropland'           : 'm01s00i448'}

# cube list
list_of_stash_ap4 = []
list_of_stash_ap5 = []

cl_ap4 = iris.load(dir + file3)
cl_ap5 = iris.load(dir + file2)

### Checking to see if STASH codes match any in the file

In [108]:
#print(c.attributes['STASH'])
i = 0
for c5 in cl_ap5:
    c5 = cl_ap5[i]
    for name, dat in stash_conFIRE.items():
        if c5.attributes["STASH"] == dat:
            print(name + " in ap5")
    i += 1

j = 0
for c4 in cl_ap4:
    c4 = cl_ap4[j]
    for name, dat in stash_conFIRE.items():
        if c4.attributes["STASH"] == dat:
            print(name + " in ap4")
    j += 1

vegcover in ap5
lightning in ap5
alpha in ap5
relative_humidity in ap5


## Variable location:

### ap4

* Can't find ``pasture`` and `cropland` here, but they may be in fractional cover, along with vegcover and treecover

### ap5

* ``vegcover``
* ``treeCover``
* ``alpha``
* ``relative_humdity``
* ``lightning``

In [98]:
i = 0
for c in cl_ap5:
    c = cl_ap5[i]
    #print(c.name())
    list_of_stash_ap5.append(c.attributes["STASH"])
    i += 1

In [109]:
for name, dat in stash_conFIRE.items():
    data = iris.load(dir + file2, iris.AttributeConstraint(STASH = dat))
    outfile = '../outputs/stash_out/' + name + '1929oct.nc'
    iris.save(data, outfile)

0: m01s03i317 / (unknown)              (pseudo_level: 27; latitude: 144; longitude: 192)
0: moisture_content_of_soil_layer / (kg m-2) (depth: 4; latitude: 144; longitude: 192)
0: m01s50i082 / (unknown)              (latitude: 144; longitude: 192)
0: relative_humidity / (%)             (latitude: 144; longitude: 192)


### This is for creating output names for when multiple years are used 

Just replace `+ name + 1929oct` with `date[k]` and set as a counter when saving the files

In [50]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'aug', 'sep', 'oct', 'nov', 'dec']
k = 0
date =[]

for i in range(1850,2015):
    for month in months:
        date.append(str(i) + month)

In [110]:
# Something handy commands you may still need and don't want to get rid of:

date
list_of_stash == stash_conFIRE.values()
list_of_stash_ap5
stash_conFIRE.values()

dict_values(['m01s03i317', 'm01s08i223', 'm01s50i082', 'm01s03i245'])